# 1:N 관계설정
- Database에서 하나의 record(행)가 다른 테이블의 여러 record가 연결될 수 있는 관계
- ex1) 하나의 유저가 여러 게시글을 썼을 때 유저의 게시글들이 유저와 연결되는 모양
- ex2) 게시글이 모여있는 DB table에서 하나의 게시글에 여러 댓글(다른 table)이 relationship을 갖게 되는 모양
![1:N](https://www.prisma.io/docs/assets/images/one-to-many-9fd7879c7dbeeb3a0409b55ba7f8ccd9.png)

## 구현방식

- 기본 키 (Primary Key): '1' 측 테이블에서 식별자 역할(1 : 한 테이블의 특정 레코드)
- 외래 키 (Foreign Key): 'N' 측 테이블에서 '1' 측의 기본 키를 참조(N: 다른 테이블의 여러 레코드)

## Django에서 코드구조

### forms.py에서

```python
class ArticleForm(ModelForm):
    class Meta():
        model = Article
        fields = '__all__'


class CommentForm(ModelForm):
    class Meta():
        model = Comment
        # fields = '__all__'

        # fields => 추가할 필드 목록
        # fields = ('content', )

        # exclude => 제외할 필드 목록
        exclude = ('article', )
```

### models.py에서

```python
class Article(models.Model):
    title = models.CharField(max_length=100)
    content = models.TextField()
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    # comment_set = 

class Comment(models.Model):
    content = models.TextField()
    article = models.ForeignKey(Article, on_delete=models.CASCADE)
    # article_id = 
```

- Article이 가진 primary key로 Comment가 접근
- 위의 구조에서 Comment는 Article이 삭제된다면 같이 삭제 되게 설정

#### on_delete

- **models.CASCADE** : 외래키를 포함하는 행도 함께 삭제
- models.PROTECT : 해당 요소가 함께 삭제되지 않도록 보호하며 오류 발생 시키기(ProtectedError)
- models.SET_NULL : 외래키 값을 NULL 값으로 변경 (null=True일 때 사용 가능)
- models.SET(func) : 외래키 값을 func 행동 수행 (func는 함수나 메서드 등을 의미)
- models.DO_NOTHING : 아무 행동을 하지 않음

### views.py에서

```python
def detail(request, id):
    article = Article.objects.get(id=id)
    comments = article.comment_set.all()
    
    form = CommentForm()

    context = {
        'article': article,
        'form': form,
        'comments': comments,
    }

    return render(request, 'detail.html', context)

def comment_create(request, article_id):
    if request.method == 'POST':
        form = CommentForm(request.POST)
        if form.is_valid():
            comment = form.save(commit=False)

            article = Article.objects.get(id=article_id)
            comment.article = article
            comment.save()

            return redirect('articles:detail', id=article_id)

    else:
        return redirect('articles:index')


def comment_delete(request, article_id, id):
    comment = Comment.objects.get(id=id)
    comment.delete()

    return redirect('articles:detail', id=article_id)
```